In [1]:
import JDM
import pandas as pd

In [2]:
t_search = JDM.Trie() 

def build_mwe(): 
    df = pd.read_csv("MWE/mwe_id_word.csv", sep=";")

    from alive_progress import alive_bar

    with alive_bar(len(df.index), force_tty=None) as bar:
        for index, row in df.iterrows():
            t_search.addword(row['words'].split())
            
            bar()

In [3]:
build_mwe()

|████████████████████████████████████████| 1738444/1738444 [100%] in 1:29.4 (19449.46/s) 


In [49]:
from alive_progress import alive_bar
from itertools import combinations
from statistics import mean, geometric_mean, harmonic_mean

def remove_useless_words(phrase):
    # Enlever la majuscule au début
    phrase = phrase[0].lower() + phrase[1:]

    # Enlever les articles
    phrase = phrase.replace(" le ", " ")
    phrase = phrase.replace(" la ", " ")
    phrase = phrase.replace(" les ", " ")
    phrase = phrase.replace(" un ", " ")
    phrase = phrase.replace(" une ", " ")
    phrase = phrase.replace(" des ", " ")
    phrase = phrase.replace(" du ", " ")
    phrase = phrase.replace(" de ", " ")

    phrase = phrase.replace("  ", " ")


    return phrase

def found_compound_terms(phrase): 
    #print(t_search.isCT([phrase[0], phrase[1]])) 

    compound_term = []
    phrase_without_compound_terms = []

    i = 0

    while i < len(phrase):
        print("for i : ", i)
        compound_term = []

        while i < len(phrase) and t_search.isCT(compound_term + [phrase[i]]): 
            print("while i : ", i)
            compound_term += [phrase[i]]
            i = i + 1

        if len(compound_term) == 0: 
            phrase_without_compound_terms += [phrase[i]]
            i = i + 1
        else : 
            new_compound_term = ""

            for term in compound_term : 
                new_compound_term += (term + " ")

            new_compound_term = new_compound_term[:len(new_compound_term)-1]

            phrase_without_compound_terms += [new_compound_term]

        

    return phrase_without_compound_terms

def relationDistance1(A, B, csv=False): 
    DWH1 = open(JDM.PATH_DWH + JDM.DWH_1, "a")

    relation = A.RS.loc[A.RS['node2'] == B.getID()]

    relation_A_B = {}

    #with alive_bar(len(relation.index), force_tty=True) as bar:
    for i, row_i in relation.iterrows() : 
        mot1 = A.getMot()
        type_relation = row_i['type']
        name_relation = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
        weight_relation = row_i['w']
        mot2 = B.getMot() 

        relation_A_B[mot1 + " " + name_relation + " " + mot2] = weight_relation

        if csv: 
            DWH1.writelines(mot1 + ";" + str(type_relation) + ";" + name_relation + ";" + str(weight_relation) + ";" + str(mot2) + "\n")

            #bar()

    DWH1.close() 

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    #for r in relation_A_B_sorted: 
        #print(str(r) + " : " + str(relation_A_B_sorted[r]))

    return relation_A_B_sorted

def relationDistance2(A, B, csv=False):
    termes = []

    #with alive_bar(len(A.RS['node2']), force_tty=True, title="Download ") as bar:
    for eidS in A.RS['node2']: 
        terme = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

        if JDM.Terme.isValidTerme(terme):
            termes.append(terme)
        #bar() 


    termes = list(dict.fromkeys(termes))
    
    DWH2 = open(JDM.PATH_DWH + JDM.DWH_2, "a")
    relation_A_B = {}

    with alive_bar(len(termes), force_tty=True, title="Search   ") as bar:
        for terme in termes:     
            AB = JDM.Terme(terme)
            relation_A_AB = A.RS.loc[A.RS['node2'] == AB.getID()]

            for i, row_i in relation_A_AB.iterrows(): 
                relation_AB_B = AB.RS.loc[AB.RS['node2'] == B.getID()]

                for j, row_j in relation_AB_B.iterrows():
                    #mot1;type_relation_1;name_relation_1;weight_relation_1;mot2;type_relation_2;name_relation_2;weight_relation_2;mot3;mean
                    mot1 = A.getMot()
                    type_relation_1 = row_i['type']
                    name_relation_1 = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
                    name_relation_1 = name_relation_1 if name_relation_1 !=  "Series([], )" else "Null"
                    weight_relation_1 = row_i['w']
                    mot2 = AB.getMot() 
                    type_relation_2 = row_j['type']
                    name_relation_2 = AB.RT.loc[AB.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False)
                    name_relation_2 = name_relation_2 if name_relation_2 !=  "Series([], )" else "Null"
                    weight_relation_2 = row_j['w']
                    mot3 = B.getMot() 
                    mean_of_relations = mean([row_i['w'], row_j['w']])
                    mean_geo_of_relations = geometric_mean([row_i['w'] if row_i['w'] > 0 else 1, row_j['w'] if row_j['w'] > 0 else 1])
                    mean_har_of_relations = harmonic_mean([row_i['w'] if row_i['w'] > 0 else 0, row_j['w'] if row_j['w'] > 0 else 0])


                    relation_A_B[mot1 + " " + name_relation_1 + " " + mot2 + " " + name_relation_2 + " " + mot3] = mean_of_relations

                    if csv: 
                        DWH2.writelines(mot1 + ";" + 
                                        str(type_relation_1) + ";" + 
                                        name_relation_1 + ";" + 
                                        str(weight_relation_1)  + ";" + 
                                        mot2 + ";" + 
                                        str(type_relation_2) + ";" + 
                                        name_relation_2 + ";" + 
                                        str(weight_relation_2) + ";" + 
                                        mot3 + ";" + 
                                        str(mean_of_relations) + ";" + 
                                        str(mean_geo_of_relations) + ";" + 
                                        str(mean_har_of_relations) + "\n")

            bar()

    DWH2.close() 

    #relation_A_B_sorted = {}

    #for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        #relation_A_B_sorted[i] = relation_A_B[i]

    #for r in relation_A_B_sorted: 
        #print(str(r) + " : " + str(relation_A_B_sorted[r]))

    return relation_A_B

def analyser(phrase): 
    # Supprimer les articles
    phrase = remove_useless_words(phrase)

    # Transformer la phrase en tableau
    phrase = phrase.split(" ")

    print("Phrase : " + str(phrase))

    # Trouver les termes composées
    phrase = found_compound_terms(phrase)
    print("Phrase : " + str(phrase)) 
    # Chercher les relations entre les termes
    list_of_term = []

    for term in phrase: 
        list_of_term.append(JDM.Terme(term))

    for term in list_of_term: 
        print(term.getMot())


    ## Test avec les combinaison
    relation = {}

    term_combination = combinations(list_of_term, 2)

    for term in list(term_combination):
        print(term[0].getMot(), term[1].getMot())
        #relation.update(relationDistance1(term[0], term[1]))
        relation.update(relationDistance2(term[0], term[1]))

    relation_sorted = {}

    for i in sorted(relation, key=relation.get, reverse=True):
        relation_sorted[i] = relation[i]

    for r in relation_sorted: 
        print(str(relation_sorted[r]) + " : " + str(r))
    

    ## Fin du test 

    

In [50]:
analyser("le chat blanc boit du lait de chèvre")

Phrase : ['le', 'chat', 'blanc', 'boit', 'lait', 'chèvre']
for i :  0
while i :  0
while i :  1
for i :  2
while i :  2
for i :  3
while i :  3
for i :  4
while i :  4
for i :  5
while i :  5
Phrase : ['le chat', 'blanc', 'boit', 'lait', 'chèvre']
le chat
blanc
boit
lait
chèvre
le chat blanc
le chat boit
le chat lait
le chat chèvre
blanc boit
blanc lait
blanc chèvre


KeyboardInterrupt: 

In [ ]:
analyser("Loic mange du caca mou")

Phrase : ['loic', 'mange', 'caca', 'mou']
for i :  0
for i :  1
while i :  1
for i :  2
while i :  2
for i :  3
while i :  3
Phrase : ['loic', 'mange', 'caca', 'mou']
loic
mange
caca
mou
loic mange
loic caca
loic mou
mange caca
mange mou
caca mou
34 : caca caractéristique mou


In [48]:
analyser("Le roman raconte l'histoire de Célestino Marcilla, un ancien combattant républicain lors de la guerre d’Espagne.")

Phrase : ['le', 'roman', 'raconte', "l'histoire", 'Célestino', 'Marcilla,', 'ancien', 'combattant', 'républicain', 'lors', 'guerre', 'd’Espagne.']
for i :  0
while i :  0
while i :  1
for i :  2
for i :  3
while i :  3
for i :  4
for i :  5
for i :  6
while i :  6
while i :  7
for i :  8
while i :  8
for i :  9
while i :  9
for i :  10
while i :  10
for i :  11
Phrase : ['le roman', 'raconte', "l'histoire", 'Célestino', 'Marcilla,', 'ancien combattant', 'républicain', 'lors', 'guerre', 'd’Espagne.']
le roman
raconte
l'histoire
Célestino
Marcilla,
ancien combattant
républicain
lors
guerre
d’Espagne.
le roman raconte
le roman l'histoire
le roman Célestino
le roman Marcilla,
le roman ancien combattant
le roman républicain
le roman lors
le roman guerre
le roman d’Espagne.
raconte l'histoire
raconte Célestino
raconte Marcilla,
raconte ancien combattant
raconte républicain
raconte lors
raconte guerre
raconte d’Espagne.
l'histoire Célestino
l'histoire Marcilla,
l'histoire ancien combattant
l'

In [ ]:
analyser("le chat blanc boit du lait de chèvre")

In [47]:
from itertools import combinations
A = ['le chat', 'blanc', 'boit', 'lait', 'chèvre']
temp = combinations(A, 2)
for i in list(temp):
	print (i)

('le chat', 'blanc')
('le chat', 'boit')
('le chat', 'lait')
('le chat', 'chèvre')
('blanc', 'boit')
('blanc', 'lait')
('blanc', 'chèvre')
('boit', 'lait')
('boit', 'chèvre')
('lait', 'chèvre')
